# The functional API

## Wide and Deep Neural Nets

<img src = "../../img/0000.png" />

In [1]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

## Loading the data

In [3]:
housing = fetch_california_housing()

In [4]:
housing.data.shape

(20640, 8)

## Train, Test & Validation set

In [5]:
x_train, x_test, y_train, y_test = train_test_split(housing.data, housing.target, test_size = 0.25)

In [6]:
x_valid, y_valid, x_train, y_train = x_train[:5000], y_train[:5000], x_train[5000:], y_train[5000:]

## Scaling Data
Scaling means that we gonna normalize the range of variables (feature space). For e.g. that all values are between 0-1

Scaling is in some machine learning tasks essentiel: 

__1. When calculating distances__ (e.g. Euclidian Distance)  
__2. When using Gradient Descent Optimizers__ (Without scaling optimizer may doesn't find optimum and optimization takes much longer..)  
__3. When Regularization is used__

In [7]:
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)
x_valid = scaler.transform(x_valid)

## Building the model (1 Input)

### First define layers  (No data processed yet)

In [8]:
input_ = keras.layers.Input(shape = x_train.shape[1:])
hidden_1 = keras.layers.Dense(30, activation=keras.activations.relu)(input_)
hidden_2 = keras.layers.Dense(30, activation = keras.activations.relu)(hidden_1)
concat = keras.layers.Concatenate()([input_, hidden_2])
out = keras.layers.Dense(1)(concat)

### Declaring the model

In [9]:
model = keras.models.Model(inputs = [input_], outputs = [out])

### Compiling & Training the model 

In [10]:
model.compile(optimizer=keras.optimizers.SGD(learning_rate = 1e-3), loss = "mse")

In [11]:
model.fit(x_train, y_train, epochs = 30)

Epoch 1/30
328/328 [==============================] - 0s 593us/step - loss: 1.6466
Epoch 2/30
328/328 [==============================] - 0s 567us/step - loss: 0.7006
Epoch 3/30
328/328 [==============================] - 0s 572us/step - loss: 0.6507
Epoch 4/30
328/328 [==============================] - 0s 590us/step - loss: 0.6225
Epoch 5/30
328/328 [==============================] - 0s 575us/step - loss: 0.6019
Epoch 6/30
328/328 [==============================] - 0s 587us/step - loss: 0.5844
Epoch 7/30
328/328 [==============================] - 0s 563us/step - loss: 0.5695
Epoch 8/30
328/328 [==============================] - 0s 544us/step - loss: 0.5568
Epoch 9/30
328/328 [==============================] - 0s 547us/step - loss: 0.5460
Epoch 10/30
328/328 [==============================] - 0s 554us/step - loss: 0.5361
Epoch 11/30
328/328 [==============================] - 0s 529us/step - loss: 0.5280
Epoch 12/30
328/328 [==============================] - 0s 550us/step - loss: 0.5205
E

In [12]:
model.evaluate(x_test, y_test)

162/162 [==============================] - 0s 454us/step - loss: 0.4475


0.4475395381450653

## Models with multiple inputs

<img src = "../../img/0001.png" />

__Notes__:  
1. We split the available features. One part goes to Input_A one part to Input_B
2. 5 features are sent to the wide path
3. 6 features are send to the deep path

In [13]:
input_a = keras.layers.Input(shape=[5])
input_b = keras.layers.Input(shape=[6])

hidden_1 = keras.layers.Dense(30, activation = keras.activations.relu)(input_b)
hidden_2 = keras.layers.Dense(30, activation = keras.activations.relu)(hidden_1)

concat = keras.layers.Concatenate()([input_a, hidden_2])

out = keras.layers.Dense(1)(concat)

model = keras.models.Model(inputs = [input_a, input_b], outputs=[out])

In [14]:
X_train_A, X_train_B = x_train[:, :5], x_train[:, 2:]
X_valid_A, X_valid_B = x_valid[:, :5], x_valid[:, 2:]
X_test_A, X_test_B = x_test[:, :5], x_test[:, 2:]
X_new_A, X_new_B = X_test_A[:3], X_test_B[:3]

In [15]:
model.compile(optimizer=keras.optimizers.SGD(learning_rate = 1e-3), loss=keras.losses.mean_squared_error)

In [16]:
model.fit((X_train_A,X_train_B), y_train, epochs = 30, validation_data = ((X_valid_A, X_valid_B), y_valid))

Epoch 1/30
328/328 [==============================] - 0s 1ms/step - loss: 2.1389 - val_loss: 0.9998
Epoch 2/30
328/328 [==============================] - 0s 844us/step - loss: 0.9437 - val_loss: 0.8081
Epoch 3/30
328/328 [==============================] - 0s 838us/step - loss: 0.7741 - val_loss: 0.7230
Epoch 4/30
328/328 [==============================] - 0s 829us/step - loss: 0.7054 - val_loss: 0.6756
Epoch 5/30
328/328 [==============================] - 0s 823us/step - loss: 0.6633 - val_loss: 0.6417
Epoch 6/30
328/328 [==============================] - 0s 810us/step - loss: 0.6330 - val_loss: 0.6164
Epoch 7/30
328/328 [==============================] - 0s 835us/step - loss: 0.6096 - val_loss: 0.5958
Epoch 8/30
328/328 [==============================] - 0s 854us/step - loss: 0.5904 - val_loss: 0.5794
Epoch 9/30
328/328 [==============================] - 0s 829us/step - loss: 0.5743 - val_loss: 0.5640
Epoch 10/30
328/328 [==============================] - 0s 813us/step - loss: 0.5603 

In [17]:
model.evaluate((X_test_A, X_test_B), y_test)

162/162 [==============================] - 0s 481us/step - loss: 0.4565


0.45645880699157715

In [18]:
model.predict((X_new_A, X_new_B))

array([[0.9824607],
       [2.414165 ],
       [2.2013783]], dtype=float32)

## Models with multiple outputs

Sometimes we also need models with multiple outputs: 

- Task demands it (locate and classifiy bounding boxes)
- Similar independent tasks based on the same data. 
- Multiclass classifications on pictur
- Regularization techniques


It's implemented equivalent to the implementation above with multiple inputs

<img src = "../../img/1016.png" />